In [ ]:
import os
import json
import hashlib
from sentence_transformers import SentenceTransformer

CARPETA_DISCURSOS = "DiscursosOriginales"
ARCHIVO_SALIDA = "discursos-final-202373580-9-202473508-1.json"

model = SentenceTransformer("all-MiniLM-L6-v2")

def calcular_sha256(texto: str) -> str:
    h = hashlib.sha256()
    h.update(texto.encode("utf-8"))
    return h.hexdigest()

def procesar_discursos():
    documentos = []

    for nombre in sorted(os.listdir(CARPETA_DISCURSOS)):
        if not nombre.lower().endswith(".txt"):
            continue

        ruta_archivo = os.path.join(CARPETA_DISCURSOS, nombre)

        with open(ruta_archivo, "r", encoding="utf-8") as f:
            texto = f.read().strip()

        _id = calcular_sha256(texto)
        embedding = model.encode(texto).tolist()

        doc = {
            "_id": _id,
            "texto": texto,
            "embedding": embedding
        }

        documentos.append(doc)
        print(f"Procesado {nombre}")

    # Guardar como JSON en forma de ARRAY
    with open(ARCHIVO_SALIDA, "w", encoding="utf-8") as out:
        json.dump(documentos, out, ensure_ascii=False, indent=2)

    print(f"\nSe guardaron {len(documentos)} documentos en {ARCHIVO_SALIDA}")

if __name__ == "__main__":
    procesar_discursos()


In [ ]:
from pymongo import MongoClient
import json

client = MongoClient(
    "mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set&readPreference=primary&ssl=false"
)

db = client["Politica"]
coleccion = db["Discursos"]

print("Conectado a MongoDB Replica Set")
print(f"Nodos disponibles: {client.nodes}")

In [ ]:
with open("discursos-final-202373580-9-202473508-1.json", "r", encoding="utf-8") as f:
    documentos = json.load(f)

resultado = coleccion.insert_many(documentos)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def buscar_discursos_similares(consulta_texto, top_k=5):

    embedding_consulta = model.encode(consulta_texto).reshape(1, -1)
    
    documentos = list(coleccion.find({}))
    

    resultados = []
    for doc in documentos:
        embedding_doc = np.array(doc['embedding']).reshape(1, -1)
        similitud = cosine_similarity(embedding_consulta, embedding_doc)[0][0]
        
        resultados.append({
            '_id': doc['_id'],
            'texto': doc['texto'],
            'similitud': similitud
        })
    

    resultados_ordenados = sorted(resultados, key=lambda x: x['similitud'], reverse=True)[:top_k]
    
    return resultados_ordenados

consulta_ejemplo = "discurso sobre la democracia y la libertad"
resultados = buscar_discursos_similares(consulta_ejemplo)

print("Top 5 discursos más similares:")
for i, resultado in enumerate(resultados, 1):
    print(f"\n--- Resultado {i} ---")
    print(f"Similitud: {resultado['similitud']:.4f}")
    print(f"Texto: {resultado['texto']}")
    print(f"ID: {resultado['_id']}")